In [ ]:
import face_recognition
import datetime
import cv2
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
from deepface import DeepFace
import os
from deepface.commons import functions

# Load known faces and their encodings
known_faces = []
known_names = []
target_size = (160, 160)
detector_backend = "opencv"
known_image1 = face_recognition.load_image_file("C:/Users/BHANU PRATAP SINGH/Desktop/Test/bhanu")
known_encoding1 = face_recognition.face_encodings(known_image1)[0]
known_faces.append(known_encoding1)
known_names.append("Bhanu")

known_image2 = face_recognition.load_image_file("C:/Users/BHANU PRATAP SINGH/Desktop/Test/ujjwal")
known_encoding2 = face_recognition.face_encodings(known_image2)[0]
known_faces.append(known_encoding2)
known_names.append("ujjwal")


# Initialize webcam
video_capture = cv2.VideoCapture(0)
# Initialize liveness detection model (OpenCV Haarcascades)
face_cascade = cv2.CascadeClassifier("C:/Users/BHANU PRATAP SINGH/Downloads/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("C:/Users/BHANU PRATAP SINGH/Downloads/haarcascade_eye.xml")

# Process frames from webcam
while True:
    ret, frame = video_capture.read()

    # Save the frame as an image
    img_path = "frame.jpg"
    cv2.imwrite(img_path, frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi_gray = gray[y:y + h, x:x + w]
        face_roi_color = frame[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(face_roi_gray)
        if len(eyes) == 0:
            # No eyes detected, likely a spoof attempt
            continue

    result = DeepFace.verify("frame.jpg", "C:/Users/BHANU PRATAP SINGH/Desktop/train/19.jpeg", detector_backend='ssd',enforce_detection=False)
    if result['verified']:
        face_distance = result['distance']
        if face_distance > 0.6:
            continue

    # Find faces in the frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    matched_names = []

    for face_encoding in face_encodings:
        # Compare face encoding with known faces
        results = face_recognition.compare_faces(known_faces, face_encoding)
        matched_indices = [i for i, result in enumerate(results) if result]
        if len(matched_indices) > 0:
            matched_names.extend([known_names[i] for i in matched_indices])
        else:
            # Unknown person
            matched_names.append("Unknown")

    # Display recognized names on the frame
    for (top, right, bottom, left), name in zip(face_locations, matched_names):
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
video_capture.release()
cv2.destroyAllWindows()
# Calculate F1 and F2 scores
y_true = ["Bhanu", "ujjwal"]
y_pred = matched_names
f1 = f1_score(y_true, y_pred, average="micro")
f2 = (5 * f1_score(y_true, y_pred, average="weighted")) / 4

# Print F1 and F2 scores
print("F1 Score:", f1)
print("F2 Score:", f2)

# Calculate confusion matrix
labels = np.unique(y_true + y_pred)
cm = confusion_matrix(y_true, y_pred, labels=labels)
print("Confusion Matrix:")
print(cm)

# Write attendance to a CSV file
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
attendance = matched_names
with open("C:/Users/BHANU PRATAP SINGH/Desktop/attendance.csv", "a") as file:
    file.write(f"Date: {current_time}\n")
    file.write("Present students: ")
    file.write(", ".join(attendance))
    file.write("\n\n")

# Print attendance
print("Attendance:")
for student in attendance:
    print(student)

